###  Simulate `build_prediction` function - Version 2

Here we mimic the process used in the Inference Detection Layer:
- pick the class with highest probability from mrcnn_class, and it's corresponding bounding box delta
- apply the deltas to the rpn_roi (output_roi)
- convert to image pixel coordinates
-?? Should we apply intra-class NMS??

In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))
%matplotlib inline
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('../')
import tensorflow as tf
import keras.backend as KB
import numpy as np
from mrcnn.datagen     import data_generator, load_image_gt
from mrcnn.callbacks   import get_layer_output_1,get_layer_output_2
from mrcnn.utils       import mask_string
import mrcnn.visualize as visualize
from mrcnn.prep_notebook import prep_newshapes_test
import matplotlib.pyplot as plt
# model_file  = 'E:\\Models\\mrcnn_logs\\shapes20180509T1928\\mask_rcnn_shapes_2192.h5'
# model_file  = 'E:\\Models\\mrcnn_logs\\TESTshapes20180511T1742\mask_rcnn_shapes_0023.h5'
# model_file  = 'E:\\Models\\mrcnn_development_logs\\shapes20180513T1946\\mask_rcnn_shapes_4044.h5'
# model_file  = 'E:\\Models\\mrcnn_oldshape_test_logs\\mask_rcnn_shapes_3877.h5'


import pprint
pp = pprint.PrettyPrinter(indent=2, width=100)

# model_file  = 'E:\\Models\\newshape_fcn\\mask_rcnn_shapes_0991.h5'
model_file  = 'E:\\Models\\newshape_fcn\\\mask_rcnn_shapes_0589.h5'
folder_name = 'newshape_fcn'
model, dataset_test, test_generator, inference_config = prep_newshapes_test(init_with = model_file, FCN_layers = True, batch_sz = 1, folder_name = folder_name)
 

In [ ]:
test_batch_x, test_batch_y = next(test_generator)

In [ ]:
# test_batch_x, test_batch_y = next(test_generator)
imgmeta_idx = model.keras_model.input_names.index('input_image_meta')
img_meta    = test_batch_x[imgmeta_idx]

for img_idx in range(inference_config.BATCH_SIZE):
    image_id = img_meta[img_idx,0]
    print('Image id: ',image_id)
    print('Image meta', img_meta[img_idx])


    image = dataset_test.load_image(image_id)
    mask, class_ids = dataset_test.load_mask(image_id)
    print('Classes (1: circle, 2: square, 3: triangle ): ',class_ids)
    visualize.display_top_masks(image, mask, class_ids, dataset_test.class_names)

In [ ]:
model.layer_info()
# model.keras_model.outputs[0].name

In [ ]:

# model_output = get_layer_output_2(model.keras_model, train_batch_x, 1)
model_output = get_layer_output_1(model.keras_model, test_batch_x, [0,1,2,3,4,5,6,7,8,9], 1)


#### Load input and output tensors

In [ ]:
input_image      =  test_batch_x[0]
input_image_meta =  test_batch_x[1]

print(' Input image shape is :', input_image.shape)
h, w = input_image.shape[1], input_image.shape[2]      #  tf.shape(input_image)[1], tf.shape(input_image)[2]

# gt_masks   =  test_batch_x[6]
print(input_image_meta)

print(len(model_output))

detections                = model_output[0]          # layer:  0   shape: (1, 100, 6)
rpn_roi_proposals         = model_output[1]          # layer:  1   shape: (1, 1000, 4)
rpn_class                 = model_output[2]          # layer:  2   shape: (1, 4092, 2)
rpn_bbox                  = model_output[3]          # layer:  3   shape: (1, 4092, 4)
mrcnn_class               = model_output[4]          # layer:  4   shape: (1, 1000, 7)
mrcnn_bbox                = model_output[5]          # layer:  5   shape: (1, 1000, 7, 4)
pred_heatmap_norm         = model_output[6]          # layer:  6   shape: (1, 128, 128, 7)
pred_heatmap_scores       = model_output[7]          # layer:  7   shape: (1, 7, 100, 11)
pred_tensor               = model_output[8]          # layer:  8   shape: (1, 7, 100, 6)
pred_heatmap              = model_output[9]          # layer:  9   shape: (1, 128, 128, 7)
# print(type(model_output[4]))
# print(type(output_rois))
for i in model_output:
    print( i.shape)
  

####  display Pred_Tensor, Pred_heatmap, mrcnn_class

In [ ]:
np.set_printoptions(precision=4, threshold=None, linewidth=210, suppress=True)
img = 0
 
print(' detections  :', detections.shape)   # 1, 100, 6
print(detections[:,:15])    # output_rois * [128, 128,128,128])
print(' pred_tensor  :', pred_tensor.shape)   # 1, 100, 6
print(pred_tensor[:,:,:5])    # output_rois * [128, 128,128,128])
print(' pred_heatmap_scores  :', pred_heatmap_scores.shape)   # 1, 100, 6
print(pred_heatmap_scores[:,:,:5])    # output_rois * [128, 128,128,128])

# print(' mrcnn_class :',mrcnn_class.shape)   # 1,1000,7
# print( mrcnn_class)
# print(' mrcnn_bbox  :', mrcnn_bbox.shape)   # 1,1000,7,4
# print( mrcnn_bbox)

# print(' rpn_roi_proposals : ', rpn_roi_proposals.shape)  # 1,1000,4
# print(' rpn_roi_proposals[:15]:', rpn_roi_proposals[:15])       #' gt_boxes[img,:15] * [128, 128,128,128])
# print(' rpn_class  : ', rpn_class.shape)                # 1,4092, 2
# print(' rpn_bbox   : ', rpn_bbox.shape)                 # 1,4092, 4 


In [ ]:
import random
# image_id2 = random.choice(dataset_test.image_ids)
image_id2 = 28
image2, image_meta2, gt_class_id2, gt_bbox2, gt_mask2 =\
    load_image_gt(dataset_test, inference_config, image_id2, use_mini_mask=False)
print(image_meta2)
print(len(image2))

# image_id = img_meta[img_idx,0]
# image = dataset_test.load_image(image_id)
mask2, class_ids2 = dataset_test.load_mask(image_id2)
print('Image id: ',image_id2)
print('Image meta', image_meta2)
print('Classes (1: circle, 2: square, 3: triangle ): ',class_ids)
visualize.display_top_masks(image2, mask2, class_ids2, dataset_test.class_names)

results = model.detect([image2], verbose=1)
print(type(results))
pp.pprint(results)

print(' detections  :', detections.shape)   # 1, 100, 6
print(detections[:,:15])  

### Prepare values to pass to build_heatmap


####  Display `output_rois` for visual check - passed on to  `build_pred_tensor()`

In [ ]:
np.set_printoptions(linewidth=150, precision=6)
# print('scatter shape is ', pred_scatt.get_shape())
print('detection shape is ', detections.shape)
img = 0
for img in range(1):
    print('Image ', img , ' ------------')
    print(detections[img,:15] )


####  Display for visual check - `pred_tensor` is the final result which is passed on to  `build_gaussian_tf()`

In [ ]:
# with sess.as_default():
np.set_printoptions(linewidth=150, precision=6)
# print('scatter shape is ', pred_scatt.get_shape())
temp = pred_tensor0
print('pred_tensor shape is ', temp.shape)
img = 0
for k in range(4):
    print('Image ', img , '/ Class ',k,' ------------')
    print(temp[img,k])


####  Display for visual check - `gt_tensor` is the final result which is passed on to  `build_gaussian_tf()`

In [ ]:
np.set_printoptions(linewidth=150, precision=6)
print('gt_tensor shape is ', gt_tensor.shape)
img = 1
for k in range(4):
    print('Image ', img , '/ Class ',k,' ------------')
    print(gt_tensor[img,k])


####  Display for visual check - `pred_heatmap_scores` is the final result which is passed from  `build_gaussian_tf()`

In [ ]:
# with sess.as_default():
np.set_printoptions(linewidth=150, precision=6, suppress=True)
# print('scatter shape is ', pred_scatt.get_shape())
print('pred_heatmap_scores shape is ', pred_heatmap_scores.shape)
for img in [0,1,2]:
    for k in range(4):
        print('Image ', img , '/ Class ',k,' ------------')
        print(np.min(pred_heatmap_scores[img,k,:,8]))
        print(pred_heatmap_scores[img,k])

####  Display for visual check - `GT_HEATMAP_SCORES` is the final result which is passed on to  `build_gaussian_tf()`

In [ ]:
np.set_printoptions(linewidth=150, precision=6)
print('gt_heatmap_scores shape is ', gt_heatmap_scores.shape)
img = 1
for k in range(4):
    print('Image ', img , '/ Class ',k,' ------------')
    print(gt_heatmap_scores[img,k])

####  Display for visual check - `FCN_HEATMAP_SCORES` is the final result which is passed on to  `build_gaussian_tf()`

In [ ]:
np.set_printoptions(linewidth=150, threshold=10000, precision=2)
temp = fcn_heatmap_scores
print('fcn_heatmap_scores shape is ', temp.shape)
# img = 1
for img in [1,2,3]:
    for k in range(4):
        print('Image ', img , '/ Class ',k,' ------------')
        print(temp[img,k])
    

####  Display for visual check - `pred_heatmap_norm` is the final result from  `build_gaussian_tf()`

In [ ]:
print(pred_heatmap_norm.shape)
temp = pred_heatmap_norm
np.set_printoptions(linewidth=150, threshold=10000, suppress=False)
print('  Temp shape :',  temp.shape)
temp_sum = np.sum(temp,axis=(1,2))
print('temp_sum is ', temp_sum.shape)
for i in range(5):
    for j in range(4):
        print('img ',i,' class ', j, ' sum:',temp_sum[i,j],  ' max: ',np.max(temp[i,:,:,j]),' mean: ', np.mean(temp[i,:,:,j]),' min: ', np.min(temp[i,:,:,j]))    


####  Display for visual check - `fcn_heatmap_norm` is the final result from  `FCN`

In [ ]:
temp = fcn_heatmap
print('  Temp shape :',  temp.shape)
temp_sum = np.sum(temp,axis=(1,2))
print('temp_sum is ', temp_sum.shape)
for i in range(5):
    for j in range(4):
        print('img / cls ',i,'/', j, ' sum:',temp_sum[i,j])
        print('sum: ', np.sum(temp[i,:,:,j] ),' max: ',np.max(temp[i,:,:,j]),' mean: ', np.mean(temp[i,:,:,j]),' min: ', np.min(temp[i,:,:,j]))    

In [ ]:
sess = KB.get_session()
with sess.as_default():
    temp = fcn_heatmap
    np.set_printoptions(linewidth=150, threshold=10000)
    print('  output shapes :',  temp.get_shape())
    temp_sum = tf.reduce_sum(temp, [2,3])
    temp_min = tf.reduce_min(temp, [2,3])
    temp_max = tf.reduce_max(temp, [2,3])
    temp_avg = tf.reduce_mean(temp, [2,3])
    print('temp_sum is ', temp_sum.shape)
    for i in range(5):
        for j in range(4):
                print('img/cls ',i,'/', j,'  sum:',temp_sum[i,j], 'min',temp_min[i,j] ,'max',temp_max[i,j] ,'avg',temp_avg[i,j])


####  Display for visual check - `gt_heatmap_scores`  and `fcn_heatmap_scores` 

In [ ]:
# with sess.as_default():
np.set_printoptions(linewidth=150, precision=6)
# print('scatter shape is ', pred_scatt.get_shape())

img = 0
for i in [1,2,3]:
    for j in range(32):
        print('Image ', img , '/ Class ',i,' ------------')

        print(gt_heatmap_scores[img,i,j])
        print(pred_heatmap_scores[img,i,j])
        print(fcn_heatmap_scores[img,i,j,[0,1,2,3,4,5,11,12,13,14,15]])

###  Display `gauss_heatmap` 3D heatmap (not normalized, normlized, L2 normalized)

In [ ]:
from mrcnn.visualize import plot_3d_heatmap, plot_2d_heatmap
%matplotlib inline
print('Image id: ',image_id , '    Classes (1: circle, 2: square, 3: triangle ): ')
img = 0
cls = 0
# for cls in range(7):
#     print(pred_heatmap_scores[img,cls,:10])

ttl = 'Non-normalized - image: {}'.format(img)
plot_3d_heatmap(pred_heatmap[img], title = ttl, width = 20)
ttl = 'Normalized - image: {}'.format(img)
plot_3d_heatmap(pred_heatmap_norm[img], title = ttl, width = 20)

# ttl = 'Non-normalized - image: {}'.format(img)
# plot_3d_heatmap(fcn_heatmap[img], title = ttl, width = 20)
# ttl = 'Normalized - image: {}'.format(img)
# plot_3d_heatmap(fcn_heatmap_norm[img], title = ttl, width = 20)

# ttl = 'Ground Truth Non-normalized - image: {}'.format(img)
# plot_3d_heatmap(gt_heatmap[img], title = ttl, width = 20)
# ttl = 'Ground Truth Normalized - image: {}'.format(img)
# plot_3d_heatmap(gt_heatmap_norm[img], title = ttl, width = 20)


# ttl = 'Non-normalized - image: {}'.format(img)
# plot_2d_heatmap(pred_heatmap[img], pred_heatmap_scores[img], width=15, title=ttl)
# ttl = 'Normalized - image: {}'.format(img)
# plot_2d_heatmap(pred_heatmap_norm[img], pred_heatmap_scores[img], width=15, title=ttl)


##  `build_predictions tensor for inferecne mode ()` 

###  Build Pred_Tensor

`detections:`

`[ 43.        63.        80.        92.         4.         0.999993]
 [ 64.         7.        78.        42.         2.         0.99999 ]
 [ 18.        38.        41.       105.         6.         0.999986]
 [ 57.        90.        70.       121.         2.         0.999873]
 [ 20.         0.        29.        40.         6.         0.999865]
 [ 44.        92.        63.       102.         5.         0.999643]
 [  0.         0.         0.         0.         0.         0.      ]`

###  Setup tensors to be passed to `build_pred_tf ()`    

In [ ]:
np.set_printoptions(linewidth=150, precision=6, threshold = 10000)
# mrcnn_bbox  = tf.identity(mrcnn_bbox)
# mrcnn_class = tf.identity(mrcnn_class)
detected_rois = tf.identity(detections)
sess = KB.get_session()
print(' New session obtained from Kras', sess)
import mrcnn.utils as utils
config = inference_config
with sess.as_default():
# def build_predictions(detection_rois, mrcnn_class, mrcnn_bbox, config):  
    batch_size      = config.BATCH_SIZE
    num_classes     = config.NUM_CLASSES
    h, w            = config.IMAGE_SHAPE[:2]
    print(' config image shape: ', config.IMAGE_SHAPE, 'h:',h,'w:',w)
    num_rois        = config.DETECTION_MAX_INSTANCES 
    num_cols        = 6
 
    print()
    print('  > build_predictions()')
    print('    num_rois          : ', num_rois )
    print('    input_rois.shape : ', KB.shape(detected_rois), KB.int_shape(detected_rois))
#     print(detected_rois.eval())
    #---------------------------------------------------------------------------
    # Build a meshgrid for image id and bbox to use in gathering of bbox delta information 
    #---------------------------------------------------------------------------
    batch_grid, bbox_grid = tf.meshgrid( tf.range(batch_size, dtype=tf.int32),
                                         tf.range(num_rois, dtype=tf.int32), indexing = 'ij' )

    #---------------------------------------------------------------------------
    # column -2 contains the prediceted class 
    #  (NOT USED)   pred_classes_exp = tf.to_float(tf.expand_dims(pred_classes ,axis=-1))    
    #---------------------------------------------------------------------------
    pred_classes = tf.to_int32(detected_rois[...,-2])
    
    gather_ind   = tf.stack([batch_grid , bbox_grid],axis = -1)
#     print(pred_classes.eval())
    
    scatter_ind = tf.stack([batch_grid , pred_classes, bbox_grid],axis = -1)
    pred_scatt  = tf.scatter_nd(scatter_ind, detected_rois, [batch_size, num_classes, num_rois, detected_rois.shape[-1]])
    print('    pred_scatter shape is ', pred_scatt.get_shape())
    
#     print(gather_ind.eval())
#     print(pred_scores.eval())
#     pred_array = input_rois
#     print('    pred_array       ', pred_array.shape)  
#     print('scatter_ind', type(scatter_ind), 'shape',tf.shape(scatter_ind).eval())
#     print(pred_scatt.eval())

    #------------------------------------------------------------------------------------
    ## sort pred_scatter in each class dimension based on sequence number (last column)
    #------------------------------------------------------------------------------------
    _, sort_inds = tf.nn.top_k(pred_scatt[...,-1], k=pred_scatt.shape[2])
    
    # build indexes to gather rows from pred_scatter based on sort order    
    class_grid, batch_grid, roi_grid = tf.meshgrid(tf.range(num_classes),tf.range(batch_size), tf.range(num_rois))
    
    roi_grid_exp = tf.to_float(tf.expand_dims(roi_grid, axis = -1))
    gather_inds  = tf.stack([batch_grid , class_grid, sort_inds],axis = -1)
    pred_tensor  = tf.gather_nd(pred_scatt, gather_inds, name = 'pred_tensor')    
    print(pred_tensor.eval())
#     return  pred_tensor    

In [ ]:
np.set_printoptions(linewidth=150, precision=6)
# print('scatter shape is ', pred_scatt.get_shape())
print('detection shape is ', detections.shape)
img = 0
for img in range(1):
    print('Image ', img , ' ------------')
    print(detections[img,:15] )

In [ ]:
np.set_printoptions(linewidth=150, threshold=10000, precision=4)
print(tst0.shape, tst1.shape, tst2.shape)
for i in [0]:
    print('---- Img/Class : ',i,'------------------')
    for j in range(32):
        for k in range(32):
            print(' output_roi: ', pred_tensor0[i,j,k])
            print('        tst: ', tst1[i,j,k])
#         print('       tst2: ', tst2[i,j])
            print()

### Print out values of `pred_tensor` , `pred_array`, ..

In [ ]:
with sess.as_default():
    scatt = pred_scatt.eval()
    test = pred_tensor.eval()

####  Test that refined_rois is correctly working in `clip_to_window_tf` 

In [ ]:
with sess.as_default():
    test_np = refined_rois.eval()
    test_tf = refined_rois_clipped.eval()
    window_np = np.array([0,0,128,128])
 
    print(window_np.shape)
    for i in range(5):
#         print('Before', i)
#         print(test_np[i])
        test_np[i] = clip_to_window(window_np, test_np[i])
#         print('After', i)
#         print(test_np[i])
#         print('   tensor flow')
#         print(test_tf[i])
        
    for i in range(5):
      all_equal = np.all(test_np == refined_rois_clipped.eval())
      print('i: ', i, '--- EQUAL : ', all_equal)


####  Test that pred_classes and pred_deltas have been properly selected when using tf.gather_nd ()

In [ ]:
with sess.as_default():
    tmp0 = pred_classes.eval()
    tmp1 = mrcnn_bbox.eval()
    tmp2 = pred_deltas.eval()
    tmp4 = mrcnn_class.eval()
    tmp3 = pred_scores2.eval()
    tmp5 = pred_scores.eval()
    

for i in range(5):
    for j in range(32):
        print('i: ', i, ' j :', j,'--- class: ',tmp0[i,j],'---------------')
    #     print(tmp0[i,j])
        print(tmp1[i,j])
        print(' ===> ', tmp2[i,j])
        print(' mrcnn_score: ', tmp4[i,j,tmp0[i,j]], ' pred_score:', tmp5[i,j,0], 'pred_score2: ', tmp3[i,j])


####  Verify refined_rois generated by TF and NP are equal when using `apply_box_deltas_tf( )`

In [ ]:
from mrcnn.utils import apply_box_deltas, apply_box_deltas_tf
with sess.as_default():
    refined_rois_tf = apply_box_deltas_3d(output_rois, pred_deltas).eval()
    print(' refined rois_tf: ', refined_rois_tf.shape, refined_rois_tf.dtype)
    tmp = []
    bxs = output_rois.eval()
    dlt = pred_deltas.eval()
    for i in range(5):
        tmp.append(apply_box_deltas(bxs[i], dlt[i]))
    refined_rois_np = np.asarray(tmp)
    print(' refined rois_np: ', refined_rois_np.shape,refined_rois_np.dtype)
    print(' refined rois_np == refined rois_tf ?? :', np.all(refined_rois_tf[0,1] == refined_rois_np[0,1]))

#     for i in range(5):
#         for j in range(32):
#             all_eq = np.all(refined_rois_tf[0,1] == refined_rois_np[0,1])
#             if ~all_eq:
#                 print(' Not equal : ',i,'/',j)
#                 print(refined_rois_tf[i,j])
#                 print(refined_rois_np[i,j])
#             else:
#                 print(' equal : ',i,'/',j)
print(refined_rois_tf[0])
print(refined_rois_np[0])


#### Window manipulation 

In [ ]:
with sess.as_default():    
    X = tf.range(18, dtype=tf.int32)
    Y = tf.range(18, dtype=tf.int32)
    X, Y = tf.meshgrid(X, Y)

    # duplicate (repeat) X and Y into a  batch_size x rois_per_image tensor
    print('    X/Y shapes :',  X.get_shape(), Y.get_shape())
    ones = tf.ones([34 , 1, 1], dtype = tf.int32)
    rep_X = ones * X
    rep_Y = ones * Y 
    print('    Ones:    ', ones.shape)                
    print('    ones_exp * X', ones.shape, '*', X.shape, '= ',rep_X.shape)
    print('    ones_exp * Y', ones.shape, '*', Y.shape, '= ',rep_Y.shape)


In [ ]:
with sess.as_default():
    
    window          = tf.constant([[0,1,2,3]], dtype =tf.float32)   
    print(window.shape)
#     high_vals = tf.stack([window[:,2],window[:,3], window[:,2], window[:,3]], axis = -1)
#     print(' hv: ', high_vals.shape)
#     print(high_vals.eval())
    high_vals = tf.expand_dims(high_vals, axis = 1)
#     print(' hv: ', high_vals.shape)
    high_vals = tf.tile(high_vals,[1,32,1])    
#     print(high_vals.shape)
#     print(high_vals[0].eval())
#     print(high_vals[1].eval())
#     print(high_vals[2].eval())

    
    
    low_vals  = tf.stack([window[:,0],window[:,1], window[:,0], window[:,1]], axis = -1)
    print(' lv: ', low_vals.shape)
    print(low_vals.eval())
    low_vals = tf.expand_dims(low_vals, axis = 1)
    print(' lv: ', low_vals.shape)
    low_vals = tf.tile(low_vals,[5, 32,1])    
    print(' lv: ', low_vals.shape)
    print(low_vals[0].eval())
    print(low_vals[1].eval())
    print(low_vals[2].eval())     
    print(low_vals[3].eval())         
    print(low_vals[4].eval())     


###  Display for visual check - `pred_tensor` is the final result which is passed on to  `build_gaussian_tf()`

In [ ]:
np.set_printoptions(linewidth=150, precision=6)
# print('scatter shape is ', pred_scatt.get_shape())
print('pred_tensor shape is ', pred_tensor.get_shape() )
img = 0
print('Image ', img , '/ Class 0 ------------')
print(pred_tensor[img,0].eval(session=sess))
print('Image ', img , '/ Class 1 ------------')
print(pred_tensor[img,1].eval(session=sess))
print('Image ', img , '/ Class 2 ------------')
print(pred_tensor[img,2].eval(session=sess))
print('Image ', img , '/ Class 3 ------------')
print(pred_tensor[img,3].eval(session=sess))


In [ ]:
 pred_tensor1  = build_predictions_tf(mrcnn_class, mrcnn_bbox, output_rois, model.config)

In [ ]:
img = 2
print(pred_tensor1[img,0].eval(session=sess))
print(pred_tensor1[img,1].eval(session=sess))
print(pred_tensor1[img,2].eval(session=sess))
print(pred_tensor1[img,3].eval(session=sess))

###  Test `means`, `covar`, `gauss_grid`, and `gauss_sum ` between development version and final version

In [ ]:
print(pred_tensor.get_shape(), pred_tensor2.get_shape())
print(np.all(pred_tensor.eval() == pred_tensor2.eval()))
# tst1 = means.eval()
# tst2 = means2.eval()
# print(tst1.shape, tst2.shape)

# print(tst1[0,:10])
# print()
# print(tst2[0,:10])
# print(np.all(tst1 == tst2))
# print()
# del tst1, tst2

In [ ]:
print(output_rois.eval())

## Proposal Layer

In [ ]:
import mrcnn.utils as utils
sess = KB.get_session()


In [ ]:
def apply_box_deltas_graph(boxes, deltas):
    """
    Applies the given deltas to the given boxes.

    x,y,w,h : Bounding Box coordinates, width, and height

    Boxes:  is the (y1, x1, y2, x2) of the anchor boxes
    deltas: [dy, dx, log(dh), log(dw)] 
            is the predicted bounding box returned from the RPN layer (in test phase)
            These are considered the targets or ground truth 
    Refer to Bounding Box Regression - R-CNN paper 
    Regression targets are calculated as follows:   
      tx = (GTx - PRx)/PRw       ty = (Gy - Py)/Ph
      th = log(Gw/Pw)         tw = log(Gh/Ph)
    ---------------------------------------------------------------------------------            
    
    boxes:  [N, 4] where each row is [y1, x1, y2, x2]
    deltas: [N, 4] where each row is [dy, dx, log(dh), log(dw)]
    """
    # Convert to y, x, h, w
    height   = boxes[:, 2] - boxes[:, 0]
    width    = boxes[:, 3] - boxes[:, 1]
    center_y = boxes[:, 0] + 0.5 * height
    center_x = boxes[:, 1] + 0.5 * width
    
    # Apply deltas
    center_y += deltas[:, 0] * height
    center_x += deltas[:, 1] * width
    height   *= tf.exp(deltas[:, 2])
    width    *= tf.exp(deltas[:, 3])
    
    # Convert back to y1, x1, y2, x2
    y1 = center_y - 0.5 * height
    x1 = center_x - 0.5 * width
    y2 = y1 + height
    x2 = x1 + width
    result = tf.stack([y1, x1, y2, x2], axis=1, name="apply_box_deltas_out")
    return result


def clip_boxes_graph(boxes, window):
    """
    clip refined anchor boxes such that they remain within the dimensions of the image 
    boxes:  [N, 4] each row is y1, x1, y2, x2
    window: [4] in the form y1, x1, y2, x2 
    """
    # Split corners
    wy1, wx1, wy2, wx2 = tf.split(window, 4)   #  0, 0 , 128,128
    y1 , x1 , y2 , x2  = tf.split(boxes, 4, axis=1)
    
    # Clip
    y1 = tf.maximum(tf.minimum(y1, wy2), wy1)   # ensure  wy1 <= y1 <= wy2
    x1 = tf.maximum(tf.minimum(x1, wx2), wx1)
    y2 = tf.maximum(tf.minimum(y2, wy2), wy1)
    x2 = tf.maximum(tf.minimum(x2, wx2), wx1)
    clipped = tf.concat([y1, x1, y2, x2], axis=1, name="clipped_boxes")
    return clipped

def suppress_small_boxes_graph(boxes, scores, area_threshold ):
    """
    supress boxes with area less than area_threshold
    boxes:  [N, 4] each row is y1, x1, y2, x2
    
    """
    bx_area = (boxes[...,2]-boxes[...,0])*(boxes[...,3]-boxes[...,1])
    selected_idxs   = tf.where(tf.greater_equal(bx_area, area_threshold))
    selected_boxes  = tf.gather_nd(boxes, selected_idxs)
    selected_scores = tf.gather_nd(scores, selected_idxs)
    padding   = tf.maximum(tf.shape(boxes)[0] - tf.shape(selected_boxes)[0], 0)

#     print(' box area       : ', bx_area.shape)    
#     print(' selected_idxs  : ', tf.shape(selected_idxs).eval())
#     print(' selected scores: ', tf.shape(selected_scores).eval())
#     print(' Req padding    : ', padding.eval())
    
    selected_boxes  = tf.pad(selected_boxes, [(0, padding), (0, 0)])    
    selected_scores = tf.pad(selected_scores, [(0, padding)])    
    return selected_boxes, selected_scores

def nms(normalized_boxes, scores):
    proposal_count = model.config.POST_NMS_ROIS_TRAINING
    rms_threshold  = model.config.RPN_NMS_THRESHOLD
    indices = tf.image.non_max_suppression(normalized_boxes, 
                                           scores, 
                                           proposal_count,
                                           nms_threshold, 
                                           name="rpn_non_max_suppression")

    proposals = tf.gather(normalized_boxes, indices)
    # Pad if needed
    padding   = tf.maximum(proposal_count - tf.shape(proposals)[0], 0)
    proposals = tf.pad(proposals, [(0, padding), (0, 0)])
    return proposals

In [ ]:
with sess.as_default():
    scores = rpn_class[:, :, 1]
    print(scores.shape)
    deltas = rpn_bbox
    print('deltas shape', deltas.shape)
    deltas = deltas * np.reshape(model.config.RPN_BBOX_STD_DEV, [1, 1, 4])
    print('deltas shape', deltas.shape)
    anchors = model.anchors
    print('model.anchors.shape', model.anchors.shape)
    pre_nms_limit = min(6000, model.anchors.shape[0])
    print('pre nms limit', pre_nms_limit)

    ix = tf.nn.top_k(scores, pre_nms_limit, sorted=True,name="top_anchors").indices

    ## gather top scores (pre_nms_limit = min(6000, # anchors) number of scores from scores)        
    scores  = utils.batch_slice([scores, ix], lambda x, y: tf.gather(x, y), model.config.IMAGES_PER_GPU) 
    print(' selected scores: ',scores.shape)

    ## get corrsponding deltas generated by RPN
    deltas  = utils.batch_slice([deltas, ix], lambda x, y: tf.gather(x, y), model.config.IMAGES_PER_GPU)
    print(' selected deltas: ',deltas.shape)
    anchors = utils.batch_slice(         ix , lambda x   : tf.gather(anchors, x), model.config.IMAGES_PER_GPU, names=["pre_nms_anchors"])
    print(' selected anchors: ',anchors.shape)
    boxes = utils.batch_slice([anchors, deltas],
                              lambda x, y: apply_box_deltas_graph(x, y),model.config.IMAGES_PER_GPU,
                              names=["refined_anchors"])
    print(' delta applied boxes :', boxes.shape)

    # Clip to image boundaries. [batch, N, (y1, x1, y2, x2)]
    height, width = model.config.IMAGE_SHAPE[:2]
    window = np.array([0, 0, height, width]).astype(np.float64)
    print(' window is ', window)
    clipped_boxes  = utils.batch_slice(boxes, 
                                   lambda x: clip_boxes_graph(x, window), model.config.IMAGES_PER_GPU,
                                   names=["refined_anchors_clipped"])

    print(' clipped boxes :', clipped_boxes.shape)
    
    ## Suppress proposal boxes (and  corresponding score) if the area is less than ROI_AREA_THRESHOLD
    roi_area_threshold = 2
    mod_boxes, mod_scores = utils.batch_slice([clipped_boxes,scores], 
                            lambda x, y: suppress_small_boxes_graph(x, y, roi_area_threshold), model.config.IMAGES_PER_GPU,
                            names=["mod_boxes", "mod_scores"])  
    print(' mod boxes :', tf.shape(mod_boxes).eval())
    print(' mod_scores:', tf.shape(mod_scores).eval())   
    
    normalized_boxes = tf.cast(mod_boxes / np.array([[height, width, height, width]]), tf.float32)
    print(' normalized boxes:', normalized_boxes.shape)

    proposals = utils.batch_slice([normalized_boxes, mod_scores], nms, model.config.IMAGES_PER_GPU)
    print('     Output: Prposals shape : ', proposals.shape, tf.shape(proposals).eval())
    

####  Analyze proposals results 

In [ ]:
sess = KB.get_session()
with sess.as_default():
    proposals = tf.identity(rpn_proposal_rois)  # <--- this uses the results from the model 

    bx_area = (proposals[...,2]-proposals[...,0])*(proposals[...,3]-proposals[...,1])
    print(' proposals :', tf.shape(proposals).eval())
    print(' box area : ', tf.shape(bx_area).eval())
    
    selected_idxs = tf.where(tf.less_equal(bx_area, (2/(128*128))) )
    print('selected bx:', tf.shape(selected_idxs).eval())
    
    print(selected_idxs.eval())
    selected_area      = tf.gather_nd(bx_area  , selected_idxs)
    selected_proposals = tf.gather_nd(proposals, selected_idxs)
    print('selected proposals shape', tf.shape(selected_proposals).eval())
    print(selected_proposals[0:30].eval())
    print('selected area shape', tf.shape(selected_area).eval())
    print(selected_area[0:30].eval())
    

####  Analyze bounding box areas 

In [ ]:
with sess.as_default():
    print(' boxes :', boxes.shape)
    for i in [0,10,17,25,26,34,39]:
        print(i, ' non-clipped ', boxes[0, i].eval())
    bx_area = (boxes[...,2]-boxes[...,0])*(boxes[...,3]-boxes[...,1])
    print(' box area : ', bx_area.shape)
    np.set_printoptions(linewidth=130,precision=4,threshold=4096)
    print(bx_area[:, :20].eval(session=sess))
    small_idxs = tf.where(bx_area < 1)
    print('small bx:', tf.shape(small_idxs).eval())
    print(small_idxs[0:10].eval())
    small_area  = tf.gather_nd(bx_area, small_idxs)
    small_boxes = tf.gather_nd(boxes, small_idxs)
    print('small boxes shape', tf.shape(small_boxes).eval())
    print(small_boxes[0:30].eval())
    print('small area shape', tf.shape(small_area).eval())
    print(small_area[0:30].eval())
    

## Detection Layer

####  Setup tensors to be passed to `detections_target_graph()`    - Detection Target Layer

In [ ]:
import mrcnn.utils  as utils
from mrcnn.detect_tgt_layer import overlaps_graph
# sess = KB.get_session()
# with  sess.as_default():
try:
    sess.close()
    print('session was deleted ')
except:
    print('Session was not defined ')
    pass
sess = tf.InteractiveSession()

image_id = 1

proposals    = KB.identity(rpn_proposal_rois)[image_id]
gt_class_ids = KB.identity(input_gt_class_ids)[image_id]
gt_boxes     = KB.cast(KB.identity(input_gt_bboxes_norm), dtype='float32')[image_id]
# gt_masks     = KB.identity(input_gt_masks)
print('rpn_roi_proposals')
print(proposals.dtype, gt_class_ids.dtype, gt_boxes.dtype)
print(proposals.shape)
print(proposals.eval())
print('gt_class_ids')
print(gt_class_ids.shape)
print(gt_class_ids.eval())
print('gt_boxes')
print(gt_boxes.shape)
print(gt_boxes.eval())
# proposals    = rpn_proposal_rois[1]
# gt_class_ids = input_gt_class_ids[1]
# gt_boxes     = input_normlzd_gt_boxes[1]
# gt_masks     = input_gt_masks[1]
# config       = model.config

### def `dev_detection_targets_graph`

In [ ]:
# def dev_detection_targets_graph(proposals, gt_class_ids, gt_boxes, gt_masks, config):
 
'''
as of 16-0-2018
Generates detection targets for one image. Subsamples proposals and
generates target class IDs, bounding box deltas, and masks for each.

Inputs:
-------
proposals:          [N, 2000, (y1, x1, y2, x2)] in normalized coordinates. 
                         Might be zero padded if there are not enough proposals.
gt_class_ids:       [MAX_GT_INSTANCES] int class IDsMx2)] in normalized coordinates.
gt_masks:           [height, width, MAX_GT_INSTANCES] of boolean type.

Returns:            Target ROIs and corresponding class IDs, bounding box shifts, and masks.
--------
rois:               [TRAIN_ROIS_PER_IMAGE, (y1, x1, y2, x2)] in normalized coordinates
class_ids:          [TRAIN_ROIS_PER_IMAGE]. Integer class IDs. Zero padded.
deltas:             [TRAIN_ROIS_PER_IMAGE, NUM_CLASSES, (dy, dx, log(dh), log(dw))]
                    Class-specific bbox refinments.
masks:              [TRAIN_ROIS_PER_IMAGE, height, width). Masks cropped to bbox
                    boundaries and resized to neural network output size.

Note: Returned arrays might be zero padded if not enough target ROIs.

''' 
# Assertions
asserts = [
    tf.Assert(tf.greater(tf.shape(proposals)[0], 0), [proposals], name="roi_assertion"),
]

with tf.control_dependencies(asserts):
    proposals = tf.identity(proposals)
# print('>>> detection_targets_graph ')
# print('     propsals.shape        :',  proposals.shape, proposals.get_shape(), KB.int_shape(proposals) )
# print('     gt_boxes.shape        :',  gt_boxes.shape ,    KB.int_shape(gt_boxes)   )
# print('     gt_class_ids.shape    :',  gt_class_ids.shape, KB.int_shape(gt_class_ids))
# print('     gt_masks.shape        :',  gt_masks.shape ,    KB.int_shape(gt_masks)   )

# Remove zero padding   
# non_zeros returns indicies to valid bboxes, which we use to index gt_class_ids, and gt_masks
proposals, non_zeros1= utils.trim_zeros_graph(proposals, name="trim_proposals")
gt_boxes, non_zeros = utils.trim_zeros_graph(gt_boxes , name="trim_gt_boxes")
gt_class_ids        = tf.boolean_mask(gt_class_ids, non_zeros, name="trim_gt_class_ids")
# gt_masks            = tf.gather(gt_masks, tf.where(non_zeros)[:, 0], axis=2,name="trim_gt_masks")

print(tf.reduce_all(non_zeros1).eval())
print(tf.shape(proposals).eval())
print(non_zeros.eval())
print(tf.shape(gt_boxes).eval())
print(gt_boxes.eval())

#------------------------------------------------------------------------------------------
# Handle COCO crowds
# A crowd box in COCO is a bounding box around several instances. Exclude
# them from training. A crowd box is given a negative class ID.
#------------------------------------------------------------------------------------------
# tf.where : returns the coordinates of true elements of  the specified conditon.
#            The coordinates are returned in a 2-D tensor where the first dimension (rows) 
#            represents the number of true elements, and the second dimension (columns) 
#            represents the coordinates of the true elements. 
#            Keep in mind, the shape of the output tensor can vary depending on how many 
#            true values there are in input. Indices are output in row-major order.
#
# tf.gather: Gather slices from params axis (default = 0) according to indices.
#            indices must be an integer tensor of any dimension (usually 0-D or 1-D). 
#            Produces an output tensor with shape:
#                   params.shape[:axis] + indices.shape + params.shape[axis + 1:] 
#
# tf.squeeze: Removes dimensions of size 1 from the shape of a tensor.
#            Given a tensor input, this operation returns a tensor of the same type with 
#            all dimensions of size 1 removed. If you don't want to remove all size 1 
#            dimensions, you can remove specific size 1 dimensions by specifying axis.
#------------------------------------------------------------------------------------------

crowd_ix        = tf.where(gt_class_ids < 0)[:, 0]
non_crowd_ix    = tf.where(gt_class_ids > 0)[:, 0]
crowd_boxes     = tf.gather(gt_boxes, crowd_ix)
# crowd_masks     = tf.gather(gt_masks, crowd_ix, axis=2)
gt_class_ids    = tf.gather(gt_class_ids, non_crowd_ix)
gt_boxes        = tf.gather(gt_boxes, non_crowd_ix)
# gt_masks        = tf.gather(gt_masks, non_crowd_ix, axis=2)
print(non_crowd_ix.eval())
print(gt_boxes.eval())

# Compute overlaps with crowd boxes [anchors, crowds]
crowd_overlaps  = overlaps_graph(proposals, crowd_boxes)
crowd_iou_max   = tf.reduce_max(crowd_overlaps, axis=1)
no_crowd_bool   = (crowd_iou_max < 0.001)

# Compute overlaps matrix [proposals, gt_boxes] - The IoU between 
# proposals and gt_boxes (non-crowd gt boxes, designated by classId < 0 in Coco)
# overlaps is 
# compute max of elements across axis 1 of overlaps tensor. 

In [ ]:
np.set_printoptions(linewidth=150, precision=6, suppress = True)
overlaps        = overlaps_graph(proposals, gt_boxes)
print(tf.shape(overlaps).eval())
print(overlaps[0:500].eval())

In [ ]:
roi_iou_max     = tf.reduce_max(overlaps, axis=1)
print('     overlaps.shape        :',  tf.shape(overlaps).eval())
print(overlaps.eval())
zero_idxs = tf.where(tf.greater_equal(overlaps,0.5))
print(tf.shape(zero_idxs).eval())

## 1. Determine indices of postive ROI propsal boxes
#    Identify ROI proposal boxes that have an IoU >= 05 overlap with some gt_box, and store 
#    indices into positive_indices
positive_roi_bool     = (roi_iou_max >= 0.5)
positive_indices      = tf.where(positive_roi_bool)[:,0]
print(tf.shape(positive_roi_bool).eval())
print(tf.shape(positive_indices).eval())
print(positive_indices.eval())



## 2. Determine indices of negative ROI proposal boxes
#    those with < 0.5 with every GT box and are not crowds bboxes 
# the where creates a array with shape [# of answers, 1] so we use [:, 0] after
## current method
negative_indices      = tf.where(tf.logical_and(roi_iou_max < 0.5, no_crowd_bool))[:, 0]

## new method
# this modification will determine negative ROI proposal boxes but in addition, 
# will suppress the zero RoIs from the indicies 
# note that   ( negative_bool         = ~positive_roi_bool)
# negative_nonzero_bool = tf.logical_and(~positive_roi_bool, (roi_iou_max > 0))
# negative_nonzero_bool = tf.logical_and(negative_nonzero_bool, no_crowd_bool)
# negative_indices2     = tf.where(negative_nonzero_bool) [:, 0]

## 3. Subsample positive ROIs based on ROI_POSITIVE_RATIO
#    Aim for 33% positive (config.ROI_POSITIVE_RATIO = 0.33)
#    Positive ROIs   33% of config.TRAIN_ROIS_PER_IMAGE ~  11
positive_count        = int(config.TRAIN_ROIS_PER_IMAGE * config.ROI_POSITIVE_RATIO)
positive_indices      = tf.random_shuffle(positive_indices)[:positive_count]
positive_count        = tf.shape(positive_indices)[0]

print(positive_indices.eval())
print(positive_count.eval())

## 4. Add Negative ROIs. Add enough to maintain positive:negative ratio
#     negative_count = int((positive_count / config.ROI_POSITIVE_RATIO) - positive_count)
r = 1.0 / config.ROI_POSITIVE_RATIO
negative_count        = tf.cast(r * tf.cast(positive_count, tf.float32), tf.int32) - positive_count
negative_indices      = tf.random_shuffle(negative_indices)[:negative_count]

## 5.   Gather selected positive and negative ROIs
positive_rois         = tf.gather(proposals, positive_indices)
negative_rois         = tf.gather(proposals, negative_indices)
print(positive_rois.eval())

## 6.   Assign positive ROIs to GT boxes.
#      roi_gt_box_assignment shows for each positive overlap, which class has the maximum overlap
positive_overlaps     = tf.gather(overlaps, positive_indices)
roi_gt_box_assignment = tf.argmax(positive_overlaps, axis=1)
roi_gt_boxes          = tf.gather(gt_boxes    , roi_gt_box_assignment)
roi_gt_class_ids      = tf.gather(gt_class_ids, roi_gt_box_assignment)

print('     shape of positive overlaps is :', positive_overlaps.get_shape())
print(roi_gt_box_assignment.eval())
print(positive_overlaps.eval())

## 7.   Compute bbox delta 
# calculate refinement (difference b/w positive rois and gt_boxes) for positive ROIs
roi_gt_deltas  = utils.box_refinement_graph(positive_rois, roi_gt_boxes)
roi_gt_deltas /= config.BBOX_STD_DEV

## 8.  prepare gt_masks 
#      transpose gt_masks from [h, w, N] to [N, height, width] and add 4th dim at end [N, height, width, 1]
#      Pick the right mask for each ROI
# transposed_masks = tf.expand_dims(tf.transpose(gt_masks, [2, 0, 1]), -1)
# roi_masks = tf.gather(transposed_masks, roi_gt_box_assignment)

# Compute mask targets
# boxes = positive_rois

# if config.USE_MINI_MASK:
    # Transform ROI corrdinates from normalized image space
    # to normalized mini-mask space.
    # y1, x1, y2, x2 = tf.split(positive_rois, 4, axis=1)
    # gt_y1, gt_x1, gt_y2, gt_x2 = tf.split(roi_gt_boxes, 4, axis=1)
    # gt_h = gt_y2 - gt_y1
    # gt_w = gt_x2 - gt_x1
    # y1 = (y1 - gt_y1) / gt_h
    # x1 = (x1 - gt_x1) / gt_w
    # y2 = (y2 - gt_y1) / gt_h
    # x2 = (x2 - gt_x1) / gt_w
    # boxes = tf.concat([y1, x1, y2, x2], 1)

# box_ids = tf.range(0, tf.shape(roi_masks)[0])
# masks   = tf.image.crop_and_resize(tf.cast(roi_masks, tf.float32), 
                                   # boxes,
                                   # box_ids,
                                   # config.MASK_SHAPE)

# Remove the extra dimension from masks.
# masks = tf.squeeze(masks, axis=3)

# Threshold mask pixels at 0.5 to have GT masks be 0 or 1 to use with
# binary cross entropy loss.
# masks = tf.round(masks)

# Append negative ROIs and pad bbox roi_gt_deltas and masks that
# are not used for negative ROIs with zeros.
rois             = tf.concat([positive_rois, negative_rois], axis=0)
N                = tf.shape(negative_rois)[0]
P                = tf.maximum(config.TRAIN_ROIS_PER_IMAGE - tf.shape(rois)[0], 0)
rois             = tf.pad(rois            , [(0, P ), (0, 0)])

roi_gt_boxes     = tf.pad(roi_gt_boxes    , [(0, N + P), (0, 0)])
roi_gt_class_ids = tf.pad(roi_gt_class_ids, [(0, N + P)])
roi_gt_deltas    = tf.pad(roi_gt_deltas   , [(0, N + P), (0, 0)])
# masks            = tf.pad(masks           , [[0, N + P], (0, 0), (0, 0)])

# print(' roi_gt_boxes :  ' , tf.shape(roi_gt_boxes) )
# print(' P:  ' , P,  ' N :    ', N)   
# print('     roi.shape             :',  rois.shape            , tf.shape(rois))
# print('     roi_gt_class_ids.shape:',  roi_gt_class_ids.shape, tf.shape(roi_gt_class_ids))
# print('     roi_gt_deltas.shape   :',  roi_gt_deltas.shape   , tf.shape(roi_gt_deltas))
# print('     masks.shape           :',  masks.shape           , tf.shape(masks))
# print('     roi_gt_boxes.shape    :',  roi_gt_boxes.shape    , tf.shape(roi_gt_boxes))

#     return rois, roi_gt_class_ids,  roi_gt_deltas, roi_gt_boxes
#     return positive_ind_shuffled, positive_indices, positive_overlaps, roi_gt_box_assignment, roi_gt_boxes, roi_gt_class_ids

In [ ]:
print(tf.shape(rois).eval())
print(rois.eval())
print(roi_gt_class_ids.eval())
print(gt_class_ids.eval())
print(gt_boxes.eval())

In [ ]:
# sess = tf.InteractiveSession()
sess = KB.get_session()
# print(input_gt_boxes[0].eval())
# print(input_normlzd_gt_boxes[0].eval())
# print(norm_output_rois[0].eval(session=sess))
